Note, still in the process of experimenting with parameters

Import packages

In [1]:
import tensorflow as tf
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pds
from tensorflow import keras
from hyperas import optim
from hyperas.distributions import choice, uniform
from hyperopt import Trials, STATUS_OK, tpe
from tensorflow.keras.optimizers import Adadelta, Nadam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

Using TensorFlow backend.


In [2]:
def create_model(x_train, y_train, x_test, y_test):
        
    model = Sequential()
    
    model.add(Dense(19, input_shape=(19,)))
    model.add(Dense({{choice([3,8,19])}}, activation={{choice(['relu', 'sigmoid', 'tanh'])}}))
    model.add(Dense({{choice([3,8,19])}}, activation={{choice(['relu', 'sigmoid', 'tanh'])}}))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mae', 
                  optimizer={{choice(['RMSProp', 'Adadelta', 'Adam', 'sgd'])}},
                  metrics=['mae', 'mape']
                 )
    
    #result = model.fit(x_train,
    #                  y_train,
    ##                  epochs = 5,
    #                  batch_size=10,
    #                  validation_data=[x_test, y_test])
    result = model.fit(x_train, y_train,
                   batch_size={{choice([64, 128])}},
                   epochs=5,
                   verbose=1,
                   validation_split=0.1)
    
    loss_metric = np.amax(result.history['mean_absolute_error']) 
    print('Current epoch:', result.history['mean_absolute_error'], result.history['mean_absolute_percentage_error'])
    return {'loss': loss_metric, 'status': STATUS_OK, 'model': model}

In [3]:
def data():
    data = pds.read_pickle('dataset_model2v2_w_label.pkl')
    dataset = data#.iloc[0:10000]
    labels = dataset['label']
    dataset = dataset.drop(columns=['label'])

    num_test_segments = 700
    msk = np.random.randint(1, dataset.journey_number.unique()[-1], num_test_segments)
    mask = dataset['journey_number'].isin(msk)

    train_input = dataset[~mask]
    test_input = dataset[mask]

    train_labels = labels[~mask]
    test_labels = labels[mask]

    train_data = train_input.drop(columns=['journey_number'])
    test_data = test_input.drop(columns=['journey_number'])

    x_train = train_input.drop(columns=['journey_number'])
    x_test = test_data
    y_train = train_labels
    y_test = test_labels
    
    return x_train, y_train, x_test, y_test

In [4]:
best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(),
                                      notebook_name="Model2v2HPO"
                                     )



>>> Imports:
#coding=utf-8

try:
    import tensorflow as tf
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import pandas as pds
except:
    pass

try:
    from tensorflow import keras
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from tensorflow.keras.optimizers import Adadelta, Nadam
except:
    pass

try:
    from tensorflow.keras import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dense': hp.choice('Dense', [3,8,19]),
        'activation': hp.choice('activation', ['relu', 'sigmoid']),
        'optimizer': hp.choice('optimizer', ['RMSProp', 'Adadelta']),
        'batch_size': hp.choice('batch_size', [64, 128]),
    }

>>> Data
  1: 
  2: data = pds.read_pickle('

Epoch 2/5
2455323/2455323 [==============================] - 17s 7us/step - loss: 13.7553 - mean_absolute_error: 13.7553 - mean_absolute_percentage_error: 117576439.3197 - val_loss: 13.6692 - val_mean_absolute_error: 13.6692 - val_mean_absolute_percentage_error: 108042717.6554
Epoch 3/5
2455323/2455323 [==============================] - 17s 7us/step - loss: 13.3673 - mean_absolute_error: 13.3673 - mean_absolute_percentage_error: 107886383.2500 - val_loss: 13.4616 - val_mean_absolute_error: 13.4616 - val_mean_absolute_percentage_error: 106881481.5051
Epoch 4/5
2455323/2455323 [==============================] - 17s 7us/step - loss: 13.2281 - mean_absolute_error: 13.2281 - mean_absolute_percentage_error: 102945290.7834 - val_loss: 13.3634 - val_mean_absolute_error: 13.3634 - val_mean_absolute_percentage_error: 97792548.2775
Epoch 5/5
2455323/2455323 [==============================] - 17s 7us/step - loss: 13.1480 - mean_absolute_error: 13.1480 - mean_absolute_percentage_error: 98707999.106

In [5]:
best_run

{'Dense': 0, 'activation': 0, 'batch_size': 0, 'optimizer': 1}

In [6]:
best_model